## Linear Regression

In [1]:
# linear regression code from scratch
import numpy as np

class LinearRegressionV0:
    def __init__(self, learning_rate=0.01, n_iterations=1000):
        self.learning_rate = learning_rate
        self.n_iterations = n_iterations
        self.weights = None
        self.bias = None

    def fit(self, X, y):
        n_samples, n_features = X.shape
        self.weights = np.zeros(n_features)
        self.bias = 0

        for _ in range(self.n_iterations):
            y_predicted = np.dot(X, self.weights) + self.bias
            dw = (1 / n_samples) * np.dot(X.T, (y_predicted - y))
            db = (1 / n_samples) * np.sum(y_predicted - y)

            self.weights -= self.learning_rate * dw
            self.bias -= self.learning_rate * db

    def predict(self, X):
        return np.dot(X, self.weights) + self.bias

In [2]:
# Sample data
X = np.array([[1, 1], [2, 2], [3, 3], [4, 4]])
y = np.array([2, 3, 4, 5])

from sklearn.datasets import make_regression
X, y = make_regression(n_samples=5, n_features=2, noise=1, random_state=42)

# Create and fit the model on sample data
model = LinearRegressionV0(learning_rate=0.01, n_iterations=1000)
model.fit(X, y)

# Make predictions
model.predict(X)

array([ 6.43090204, 37.72682388, -8.9644079 , -0.08918361, 42.31392336])

In [3]:
# Use sklearn
from sklearn.linear_model import LinearRegression

# Create and fit the model on sample data
model2 = LinearRegression()
model2.fit(X, y)

# Make predictions
model2.predict(X)

array([ 6.06623392, 38.03820354, -9.359774  , -0.16495742, 42.3063024 ])

### R-squared

**R-squared ($R^{2}$) is the percentage of variation explained by the relationship between two variables.**

$$R^{2} = \frac{Var(mean) - Var(fit)}{Var(mean)} = \frac{SS(mean) - SS(fit)}{SS(mean)}$$

If we assume that Var(mean) ≥ Var(fit), this makes $R^{2} \in [0, 1]$. This makes $R^{2}$ a percentage.

**Interpretation:**

$R^{2}$ answers how much better the line fit the data better than the mean. R-squared = 0.6 means 60% reduction in variance once we took a feature into account.

If $R^{2}$ = 0.81, then this means that there is 81% less variation around the line than the mean. In other words, the X-Y relationship accounts for 81% variation of total variation.

The statistically significant R-squared was 0.9 equals the relationship between the two variables explains 90% of the variation in the data.

**Relationship with Correlation R**

In the case of simple linear regression with only one independent variable, $R^{2}$ is just square of correlation R. Correlation $R \in [-1, 1]$, so the range for $R^{2}$ makes sense.

For example, if R = 0.9 then R-squared = 0.81 and if R = 0.5 then R-squared = 0.25.

**Why use $R^{2}$?**

$R^{2}$ is more interpretable than R because $R^{2} = 0.7$ is 1.4 times as good as $R^{2} = 0.5$.

### Adjusted R-squared

Why use adjusted R-squared? Equations with more parameters will never make SS(fit) worse than equations with fewer parameters. This is because least squares will cause any term that makes SS(fit) worse to be multiplied by 0, in a sense, no longer exist.

The more parameters we add to the equation, the more opportunities we have for random events to reduce SS(fit) and result in a better R-squared. Thus, people report an "adjusted R-square" value that, in essence, scales R-squared by the number of parameters.

$$R^{2}_{adj} = 1 - \frac{SS(fit) / (n - p - 1)}{SS(mean) / (n - 1)} = 1 - \frac{(n - 1)SS(fit)}{(n - p - 1)SS(mean)} =  1 -  \frac{n - 1}{n - p - 1} (1 - R^{2})$$

### Statistical Significance

If there is only two measurements, $R^{2} = 1$.

The p-value for $R^{2}$ comes from something called "F".

$$F = \frac{\text{The variation in Y explained by X}}{\text{The variation in Y not explained by X}} = \frac{SS(mean) - SS(fit) / (p_{fit} - p_{mean})}{SS(fit) / (n - p_{fit})}$$

How doe we turn this number into a p-value?

Generate a set of random data calculate F and repeat for many times (ex. 10,000). The p-value is number of more extreme values divided by all the values.

In [4]:
# Write a function to calculate R^2
def r_squared(y_true, y_pred):
    ss_mean = np.sum((y_true - np.mean(y_true)) ** 2)
    ss_fit = np.sum((y_true - y_pred) ** 2)
    r2 = (ss_mean - ss_fit) / ss_mean
    return r2

# Calculate R^2
print(f"R-squared (calculated):{r_squared(y, model.predict(X))}")

# R^2 from sklearn
print(f"R-squared (sklearn): {model2.score(X, y)}")

R-squared (calculated):0.9990510005866138
R-squared (sklearn): 0.9992307969801115


In [5]:
# Write a function to calculate adjusted R^2
def adjusted_r_squared(y_true, y_pred, n_features):
    n_samples = len(y_true)
    r2 = r_squared(y_true, y_pred)
    adj_r2 = 1 - ((1 - r2) * (n_samples - 1) / (n_samples - n_features - 1))
    return adj_r2

# Calculate adjusted R-squared
print(f"Adjusted R-squared (calculated): {adjusted_r_squared(y, model.predict(X), X.shape[1])}")

# Adjusted R-squared does not exist in sklearn
print(f"Adjusted R-squared (sklearn): {adjusted_r_squared(y, model2.predict(X), X.shape[1])}")

Adjusted R-squared (calculated): 0.9981020011732276
Adjusted R-squared (sklearn): 0.9984615939602228


In [6]:
# Write a function to calculate the F-statistic
def f_statistic(y_true, y_pred, n_features):
    ss_mean = np.sum((y_true - np.mean(y_true)) ** 2)
    ss_fit = np.sum((y_true - y_pred) ** 2)

    explained_variance = (ss_mean - ss_fit) / (n_features - 1)
    unexplained_variance = ss_fit / (len(y_true) - n_features)

    f_stat = explained_variance / unexplained_variance
    return f_stat


# Calculate F-statistic
print(f"F-statistic (calculated): {f_statistic(y, model.predict(X), X.shape[1])}")
print(f"F-statistic (model2): {f_statistic(y, model2.predict(X), X.shape[1])}")

# F-statistic from sklearn
from sklearn.feature_selection import f_regression
f_stat, p_value = f_regression(X, y)
print(f"F-statistic (sklearn): {f_stat}")
print(f"p-values: {p_value}")


F-statistic (calculated): 3158.224293379773
F-statistic (model2): 3897.1406942406193
F-statistic (sklearn): [9.24581414 5.40663939]
p-values: [0.05583673 0.10259428]


In [7]:
# Using statsmodels
import statsmodels.api as sm

X = sm.add_constant(X)
model = sm.OLS(y, X)
results = model.fit()
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.999
Model:                            OLS   Adj. R-squared:                  0.998
Method:                 Least Squares   F-statistic:                     1299.
Date:                Sat, 01 Feb 2025   Prob (F-statistic):           0.000769
Time:                        23:08:55   Log-Likelihood:                -4.3640
No. Observations:                   5   AIC:                             14.73
Df Residuals:                       2   BIC:                             13.56
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.8504      0.528     -1.611      0.2

/Users/sungwankim/data_science/statistics/.venv/lib/python3.11/site-packages/statsmodels/stats/stattools.py:74: ValueWarning: omni_normtest is not valid with less than 8 observations; 5 samples were given.
  warn("omni_normtest is not valid with less than 8 observations; %i "
